# Desafio Kaggle — Classificação de Emoções em Texto

> Notebook final de apresentação do trabalho da disciplina de IA (UC14).

## Objetivo
- Treinar e avaliar modelos para classificar emoções em textos curtos.
- Gerar arquivo de submissão no formato exigido pelo Kaggle.
- Documentar o pipeline que obteve melhor desempenho (**fine-tuning do ProsusAI/finbert**).

## Estrutura da apresentação
1. Setup do ambiente
2. Definição do modelo e pipeline de dados
3. Treinamento do melhor modelo
4. Avaliação e inferência
5. Geração da submissão Kaggle

In [ ]:
%pip install -q evaluate

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from datasets import load_dataset
from pathlib import Path
import numpy as np
import evaluate
import csv

def get_best_available_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    if torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

device = get_best_available_device()
print(f"Device selecionado: {device}")

## 2) Pipeline de Dados e Treinamento (Modelo Campeão)

Nesta etapa preparamos os dados, tokenizamos os textos, configuramos métricas e realizamos o fine-tuning do **ProsusAI/finbert**.

In [ ]:
# @title Define Data Pipeline (Emotion CSV)

from pathlib import Path

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def resolve_train_csv(root: Path) -> Path:
    kaggle_path = Path("/kaggle/input/datasets/pablohenriquelemes/emotion-classification-uc14/train_large.csv")
    local_candidates = [
        root / "data" / "kaggle_emotion_classification" / "train_large.csv",
        root / "train_large.csv",
    ]
    if kaggle_path.exists():
        return kaggle_path
    for candidate in local_candidates:
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Não foi possível localizar train_large.csv.")

project_root = Path.cwd()
if not (project_root / "data").exists() and (project_root.parent / "data").exists():
    project_root = project_root.parent

train_csv = resolve_train_csv(project_root)
print(f"Arquivo de treino: {train_csv}")

raw_dataset = load_dataset("csv", data_files={"train": str(train_csv)})["train"]
raw_dataset = raw_dataset.class_encode_column("label")
dataset = raw_dataset.train_test_split(test_size=0.2, seed=42, stratify_by_column="label")

num_labels = raw_dataset.features["label"].num_classes
print(f"Total de classes: {num_labels}")

train_labels_np = np.array(dataset["train"]["label"])
val_labels_np = np.array(dataset["test"]["label"])
train_counts = np.bincount(train_labels_np, minlength=num_labels)
val_counts = np.bincount(val_labels_np, minlength=num_labels)
print("Distribuição de classes (train):", train_counts.tolist())
print("Distribuição de classes (val):  ", val_counts.tolist())

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=128)

columns_to_remove = [col for col in dataset["train"].column_names if col != "label"]
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=columns_to_remove)

In [ ]:
# @title Training Setup (ProsusAI/finbert)

is_cuda = torch.cuda.is_available()
is_mps = torch.backends.mps.is_available()

def multiclass_log_loss(logits, labels, eps=1e-15):
    probs = np.exp(logits - logits.max(axis=1, keepdims=True))
    probs = probs / probs.sum(axis=1, keepdims=True)
    probs = np.clip(probs, eps, 1.0 - eps)
    one_hot = np.eye(probs.shape[1])[labels]
    return float(-(one_hot * np.log(probs)).sum(axis=1).mean())

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    ll = multiclass_log_loss(logits, labels)

    precision_weighted = precision_metric.compute(
        predictions=preds, references=labels, average="weighted"
    )["precision"]
    recall_weighted = recall_metric.compute(
        predictions=preds, references=labels, average="weighted"
    )["recall"]
    f1_weighted = f1_metric.compute(
        predictions=preds, references=labels, average="weighted"
    )["f1"]

    precision_macro = precision_metric.compute(
        predictions=preds, references=labels, average="macro"
    )["precision"]
    recall_macro = recall_metric.compute(
        predictions=preds, references=labels, average="macro"
    )["recall"]
    f1_macro = f1_metric.compute(
        predictions=preds, references=labels, average="macro"
    )["f1"]

    return {
        "accuracy": acc,
        "log_loss": ll,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
    }

best_model = AutoModelForSequenceClassification.from_pretrained(
    "ProsusAI/finbert",
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

training_args = TrainingArguments(
    output_dir="./distilbert_emotion",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_log_loss",
    greater_is_better=False,
    lr_scheduler_type="linear",
    weight_decay=0.03,
    logging_steps=50,
    dataloader_num_workers=2,
    dataloader_pin_memory=False if is_mps else True,
    remove_unused_columns=True,
    report_to="none",
)

trainer = Trainer(
    model=best_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

print(f"Device profile -> CUDA: {is_cuda} | MPS: {is_mps}")
print(f"✅ Modelo carregado: {best_model.num_parameters():,} parâmetros")
print("Métricas de validação por época: accuracy, log_loss, precision/recall/f1 (weighted e macro)")

In [ ]:
# @title Start training

trainer.model.to(device)
print(f"Treinando em: {device}")

if device.type == "mps" and hasattr(torch, "mps"):
    torch.mps.empty_cache()

trainer.train()

In [ ]:
# @title Save trained model

save_directory = "./distilbert_emotion/final_checkpoint"

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Checkpoint salvo em: {save_directory}")

## 4) Avaliação, Inferência e Submissão Kaggle

Com o melhor checkpoint treinado, avaliamos métricas no conjunto de validação e geramos o arquivo `submission.csv` no formato do desafio.

In [ ]:
# @title Load best model checkpoint

project_root = Path.cwd()
if not (project_root / "data").exists() and (project_root.parent / "data").exists():
    project_root = project_root.parent

candidate_dirs = [
    project_root / "distilbert_emotion" / "final_checkpoint",
    project_root / "notebooks" / "distilbert_emotion" / "final_checkpoint",
]

save_directory = None
for c in candidate_dirs:
    if c.exists():
        save_directory = str(c)
        break

if save_directory is None:
    raise FileNotFoundError(
        "Nenhum checkpoint final encontrado em distilbert_emotion/final_checkpoint."
    )

print(f"Carregando checkpoint de: {save_directory}")
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [ ]:
# @title Get validation metrics (relatório completo)

eval_trainer = Trainer(
    model=loaded_model,
    args=TrainingArguments(output_dir="./eval_output", report_to="none"),
    eval_dataset=tokenized_datasets["test"],
    processing_class=loaded_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

metrics = eval_trainer.evaluate()

print("\n📊 RELATÓRIO DE VALIDAÇÃO")
print(f"Validation Loss        : {metrics['eval_loss']:.5f}")
print(f"Validation Log Loss    : {metrics['eval_log_loss']:.5f}")
print(f"Validation Accuracy    : {metrics['eval_accuracy']:.2%}")
print(f"Precision (weighted)   : {metrics['eval_precision_weighted']:.2%}")
print(f"Recall (weighted)      : {metrics['eval_recall_weighted']:.2%}")
print(f"F1-score (weighted)    : {metrics['eval_f1_weighted']:.2%}")
print(f"Precision (macro)      : {metrics['eval_precision_macro']:.2%}")
print(f"Recall (macro)         : {metrics['eval_recall_macro']:.2%}")
print(f"F1-score (macro)       : {metrics['eval_f1_macro']:.2%}")

In [ ]:
# @title Run inference on new data

label_map = {0: "Sadness", 1: "Joy", 2: "Love", 3: "Anger", 4: "Fear", 5: "Surprise"}

def predict_emotion(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=False)
    inputs = {k: v.to(loaded_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = loaded_model(**inputs)
        logits = outputs["logits"]
        predicted_class_id = torch.argmax(logits, dim=-1).item()

    return label_map[predicted_class_id]

print("\n🤖 INFERENCE DEMO:")
sample_text = "I am very happy with the results of this project!"
prediction = predict_emotion(sample_text)

print(f"📝 Text: '{sample_text}'")
print(f"🏷️ Predicted Emotion: {prediction}")

In [ ]:
# @title Generate Kaggle submission (test CSV -> class probabilities)

def _resolve_test_csv(root: Path) -> Path:
    kaggle_path = Path("/kaggle/input/datasets/pablohenriquelemes/emotion-classification-uc14/test.csv")
    local_candidates = [
        root / "data" / "kaggle_emotion_classification" / "test.csv",
        root / "test.csv",
    ]
    if kaggle_path.exists():
        return kaggle_path
    for candidate in local_candidates:
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Arquivo de teste não encontrado (test.csv).")

def _load_test_rows(csv_path: Path):
    with csv_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        if not reader.fieldnames:
            raise ValueError("CSV de teste sem cabeçalho.")
        fieldnames = [name.strip() for name in reader.fieldnames]
        id_field = "id" if "id" in fieldnames else None
        text_field = "text" if "text" in fieldnames else None
        if text_field is None:
            non_id_fields = [name for name in fieldnames if name != "id"]
            if not non_id_fields:
                raise ValueError("CSV de teste precisa de uma coluna de texto.")
            text_field = non_id_fields[0]

        ids, texts = [], []
        for idx, row in enumerate(reader):
            row_id = row.get(id_field) if id_field else None
            ids.append(row_id if row_id is not None else str(idx))
            texts.append(row.get(text_field, ""))

    return ids, texts

def _batched(items, batch_size):
    for start in range(0, len(items), batch_size):
        yield items[start:start + batch_size]

project_root = Path.cwd()
if not (project_root / "data").exists() and (project_root.parent / "data").exists():
    project_root = project_root.parent

test_csv = _resolve_test_csv(project_root)
ids, texts = _load_test_rows(test_csv)

loaded_model = loaded_model.to(device)
loaded_model.eval()

all_probs = []
batch_size = 128

with torch.no_grad():
    for batch_texts in _batched(texts, batch_size):
        inputs = loaded_tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = loaded_model(**inputs)
        logits = outputs["logits"]
        probs = torch.softmax(logits, dim=-1).cpu().numpy()
        all_probs.extend(probs)

all_probs = np.asarray(all_probs, dtype=np.float64)
if all_probs.shape[1] != num_labels:
    raise ValueError(f"Número de colunas de probabilidade ({all_probs.shape[1]}) diferente de num_labels ({num_labels}).")

row_sums = all_probs.sum(axis=1)
if not np.allclose(row_sums, 1.0, atol=1e-6):
    raise ValueError("As probabilidades não somam 1.0 em todas as linhas.")

if np.isnan(all_probs).any() or np.isinf(all_probs).any():
    raise ValueError("Foram encontrados NaN/Inf nas probabilidades.")

expected_num_classes = 6
if num_labels != expected_num_classes:
    raise ValueError(
        f"Esse desafio espera {expected_num_classes} classes, mas o pipeline detectou {num_labels}."
    )
submission_class_columns = [f"prob_{i}" for i in range(expected_num_classes)]

submission_path = project_root / "submission.csv"
with submission_path.open("w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id"] + submission_class_columns)
    for row_id, prob_row in zip(ids, all_probs):
        writer.writerow([row_id] + [f"{p:.8f}" for p in prob_row])

print(f"Arquivo de submissão gerado: {submission_path}")
print(f"Total de linhas: {len(all_probs)}")
print("Header da submissão:", ["id"] + submission_class_columns)